In [65]:
#CS221_11292018_0824 includes a RAKE class that applies RAKE on the input file
import tensorflow as tf
import pandas as pd
import json
import numpy as np
import sys
import collections
import gzip
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize 
from nltk.stem import WordNetLemmatizer 



In [10]:
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield eval(l)

meta_data = []
# meta_path= '/Users/srishti/Desktop/meta_Books.json.gz'
meta_path= '/Users/srishti/Desktop/meta_Health_and_Personal_Care.json.gz'
counter= 1


for line in parse(meta_path):
    meta_data.append(line)
asin_title_map = collections.defaultdict(list)

for i in meta_data:
    if 'title' in i.keys():
        asin_title_map[i['asin']] = i['title']
    else:
        asin_title_map[i['asin']] = None


In [13]:
counter=0
for i in asin_title_map.keys():
    if counter <5:
        print i+ " "+ asin_title_map[i]
        counter+=1
        

B00CQ7SOQC Natural Vitality - Energy 28 Nutri Packs 15pk box
B0047XXSV4 Prevail Specialty Briefs, 2X-Large (Case of 48 Briefs)
B009HT2XBO 1/2&quot; + 5/8&quot; + 3/4&quot; Medical Stainless Ben Wa Ball Kegel Vaginal Tight Exerciser
B001E7666A Aura Cacia Room and Body Mist, Sensual Cinnamon and Ylang Ylang, 4 Fluid Ounce
B00BA3G7RQ Arctic Powder Super-flex Cold Pack with Magnetic Therapy, 2 Pack


In [171]:
# number_of_data_points_to_read= 10000 #Select number of data points to read
counter= 1
data=[]
# path= '/Users/srishti/Desktop/reviews_Books_5.json'reviews_Health_and_Personal_Care_5
path= '/Users/srishti/Desktop/reviews_Health_and_Personal_Care_5.json'
with open(path) as f:
    for line in f:
#         if counter <number_of_data_points_to_read:
        data.append(json.loads(line))
        counter+=1
#         else: 
#             break
   

In [172]:
print len(data)


346355


In [173]:
# path= '/Users/srishti/Google Drive/000_7th Quarter/CS221/Project/untouched_data/reviews_Automotive_5.json'
# data = []
# with open(path) as f:
#     for line in f:
#         data.append(json.loads(line))

# d[1]
# for j in data[1]:
#     print j
# print data[1]['reviewText']

X=[] #reivews
Y=[] #ratings

for i in data:
    X.append(i['reviewText'])
    Y.append(i['overall'])

print X[1]
print Y[1]


I would recommend this for a travel magnifier for the occasional reading.I had read on another review about a magnifier having a problem with the light coming on. I did find that this one appeared to be DOA out of the box. But, after opening & shutting the viewer to turn on & off the light, the light began to come on. After several times of doing this, the light appears to be coming on all the time.It is small, but for taking it someplace & reading things like a menu in a dark corner of a restaurant, this is great.
4.0


In [174]:
asin_dict = collections.defaultdict(list)
for i in data:
    X.append(i['reviewText'])
    Y.append(i['overall'])
    asin_dict[i['asin']].append(i['reviewText'])
#     if i['overall'] >= 4:
#         asin_dict[i['asin']+"_pos"].append(i['reviewText'])
        
#     else:
#         asin_dict[i['asin']+"_neg"].append(i['reviewText'])

# Merges the list of reviews for each asin_pos and asin_neg into a single string
for j in asin_dict.keys():
	asin_dict[j] = " ".join(asin_dict[j])

In [175]:
print len(asin_dict.keys())

18534


In [176]:
count=0
for i in asin_dict.keys():
    if count < 10:
        print i
        count+=1
    else:
        break
    

B000G6480G
B0013J90AS
B000EXS6D6
B0032DP7Y6
B004W4MNSM
B006OC3JY2
B006W9QIM2
B0014UFVBI
B00C1C23FU
B000RQ2DMK


In [177]:
counter=0
for i in asin_dict.keys():
#     print i 
    if i in asin_title_map.keys():
#         print i
        counter +=1    
print counter

18534


In [178]:
#
embeddings_index = dict()
f = open('/Users/srishti/Google Drive/000_7th Quarter/CS221/Project/untouched_data/glove.6B/glove.6B.50d.txt')
for line in f:# first element of each line is the word and remaining elements are numerical represenation of each line 
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs# creates a dict of words and numerical representation of that word
f.close()



In [179]:
embeddings_index['dog']
embeddings_index['time']

array([ 0.02648  ,  0.33737  ,  0.065667 , -0.11609  ,  0.41651  ,
       -0.21142  , -0.69582  ,  0.2822   , -0.36077  , -0.13822  ,
        0.012094 ,  0.086227 , -0.84638  ,  0.057195 ,  1.1582   ,
        0.14703  , -0.0049197, -0.24899  , -0.96014  , -0.3038   ,
        0.23972  ,  0.21058  ,  0.40608  ,  0.17789  ,  0.55253  ,
       -1.6357   , -0.17784  , -0.45222  ,  0.45805  ,  0.14239  ,
        3.7087   ,  0.40289  , -0.4083   , -0.29304  ,  0.030857 ,
       -0.15361  ,  0.10607  ,  0.63397  ,  0.12397  , -0.25349  ,
       -0.10344  ,  0.0069768, -0.17328  ,  0.35536  , -0.46369  ,
        0.15285  ,  0.41475  , -0.3398   , -0.23043  ,  0.19069  ],
      dtype=float32)

In [180]:
# -*- coding: utf-8 -*-
"""Implementation of Rapid Automatic Keyword Extraction algorithm.
As described in the paper `Automatic keyword extraction from individual
documents` by Stuart Rose, Dave Engel, Nick Cramer and Wendy Cowley.
"""

import string
from collections import Counter, defaultdict
from itertools import chain, groupby, product

import nltk
from enum import Enum
from nltk.tokenize import wordpunct_tokenize


class Metric(Enum):
    """Different metrics that can be used for ranking."""

    DEGREE_TO_FREQUENCY_RATIO = 0  # Uses d(w)/f(w) as the metric
    WORD_DEGREE = 1  # Uses d(w) alone as the metric
    WORD_FREQUENCY = 2  # Uses f(w) alone as the metric


class Rake(object):
    """Rapid Automatic Keyword Extraction Algorithm."""

    def __init__(
        self,
        stopwords=None,
        punctuations=None,
        language="english",
        ranking_metric=Metric.DEGREE_TO_FREQUENCY_RATIO,
        max_length=10000,
        min_length=1,
    ):
        """Constructor.
        :param stopwords: List of Words to be ignored for keyword extraction.
        :param punctuations: Punctuations to be ignored for keyword extraction.
        :param language: Language to be used for stopwords
        :param max_length: Maximum limit on the number of words in a phrase
                           (Inclusive. Defaults to 100000)
        :param min_length: Minimum limit on the number of words in a phrase
                           (Inclusive. Defaults to 1)
        """
        # By default use degree to frequency ratio as the metric.
        if isinstance(ranking_metric, Metric):
            self.metric = ranking_metric
        else:
            self.metric = Metric.DEGREE_TO_FREQUENCY_RATIO

        # If stopwords not provided we use language stopwords by default.
        self.stopwords = stopwords
        if self.stopwords is None:
            self.stopwords = nltk.corpus.stopwords.words(language)

        # If punctuations are not provided we ignore all punctuation symbols.
        self.punctuations = punctuations
        if self.punctuations is None:
            self.punctuations = string.punctuation

        # All things which act as sentence breaks during keyword extraction.
        self.to_ignore = set(chain(self.stopwords, self.punctuations))

        # Assign min or max length to the attributes
        self.min_length = min_length
        self.max_length = max_length

        # Stuff to be extracted from the provided text.
        self.frequency_dist = None
        self.degree = None
        self.rank_list = None
        self.ranked_phrases = None

    def extract_keywords_from_text(self, text):
        """Method to extract keywords from the text provided.
        :param text: Text to extract keywords from, provided as a string.
        """
        sentences = nltk.tokenize.sent_tokenize(text)
        self.extract_keywords_from_sentences(sentences)

    def extract_keywords_from_sentences(self, sentences):
        """Method to extract keywords from the list of sentences provided.
        :param sentences: Text to extraxt keywords from, provided as a list
                          of strings, where each string is a sentence.
        """
        phrase_list = self._generate_phrases(sentences)
        self._build_frequency_dist(phrase_list)
        self._build_word_co_occurance_graph(phrase_list)
        self._build_ranklist(phrase_list)

    def get_ranked_phrases(self):
        """Method to fetch ranked keyword strings.
        :return: List of strings where each string represents an extracted
                 keyword string.
        """
        return self.ranked_phrases

    def get_ranked_phrases_with_scores(self):
        """Method to fetch ranked keyword strings along with their scores.
        :return: List of tuples where each tuple is formed of an extracted
                 keyword string and its score. Ex: (5.68, 'Four Scoures')
        """
        return self.rank_list

    def get_word_frequency_distribution(self):
        """Method to fetch the word frequency distribution in the given text.
        :return: Dictionary (defaultdict) of the format `word -> frequency`.
        """
        return self.frequency_dist

    def get_word_degrees(self):
        """Method to fetch the degree of words in the given text. Degree can be
        defined as sum of co-occurances of the word with other words in the
        given text.
        :return: Dictionary (defaultdict) of the format `word -> degree`.
        """
        return self.degree

    def _build_frequency_dist(self, phrase_list):
        """Builds frequency distribution of the words in the given body of text.
        :param phrase_list: List of List of strings where each sublist is a
                            collection of words which form a contender phrase.
        """
        self.frequency_dist = Counter(chain.from_iterable(phrase_list))

    def _build_word_co_occurance_graph(self, phrase_list):
        """Builds the co-occurance graph of words in the given body of text to
        compute degree of each word.
        :param phrase_list: List of List of strings where each sublist is a
                            collection of words which form a contender phrase.
        """
        co_occurance_graph = defaultdict(lambda: defaultdict(lambda: 0))
        for phrase in phrase_list:
            # For each phrase in the phrase list, count co-occurances of the
            # word with other words in the phrase.
            #
            # Note: Keep the co-occurances graph as is, to help facilitate its
            # use in other creative ways if required later.
            for (word, coword) in product(phrase, phrase):
                co_occurance_graph[word][coword] += 1
        self.degree = defaultdict(lambda: 0)
        for key in co_occurance_graph:
            self.degree[key] = sum(co_occurance_graph[key].values())

    def _build_ranklist(self, phrase_list):
        """Method to rank each contender phrase using the formula
              phrase_score = sum of scores of words in the phrase.
              word_score = d(w)/f(w) where d is degree and f is frequency.
        :param phrase_list: List of List of strings where each sublist is a
                            collection of words which form a contender phrase.
        """
        self.rank_list = []
        for phrase in phrase_list:
            rank = 0.0
            for word in phrase:
                if self.metric == Metric.DEGREE_TO_FREQUENCY_RATIO:
                    rank += 1.0 * self.degree[word] / self.frequency_dist[word]
                elif self.metric == Metric.WORD_DEGREE:
                    rank += 1.0 * self.degree[word]
                else:
                    rank += 1.0 * self.frequency_dist[word]
            self.rank_list.append((rank, " ".join(phrase)))
        self.rank_list.sort(reverse=True)
        self.ranked_phrases = [ph[1] for ph in self.rank_list]

    def _generate_phrases(self, sentences):
        """Method to generate contender phrases given the sentences of the text
        document.
        :param sentences: List of strings where each string represents a
                          sentence which forms the text.
        :return: Set of string tuples where each tuple is a collection
                 of words forming a contender phrase.
        """
        phrase_list = set()
        # Create contender phrases from sentences.
        for sentence in sentences:
            word_list = [word.lower() for word in wordpunct_tokenize(sentence)]
            phrase_list.update(self._get_phrase_list_from_words(word_list))
        return phrase_list

    def _get_phrase_list_from_words(self, word_list):
        """Method to create contender phrases from the list of words that form
        a sentence by dropping stopwords and punctuations and grouping the left
        words into phrases. Only phrases in the given length range (both limits
        inclusive) would be considered to build co-occurrence matrix. Ex:
        Sentence: Red apples, are good in flavour.
        List of words: ['red', 'apples', ",", 'are', 'good', 'in', 'flavour']
        List after dropping punctuations and stopwords.
        List of words: ['red', 'apples', *, *, good, *, 'flavour']
        List of phrases: [('red', 'apples'), ('good',), ('flavour',)]
        List of phrases with a correct length:
        For the range [1, 2]: [('red', 'apples'), ('good',), ('flavour',)]
        For the range [1, 1]: [('good',), ('flavour',)]
        For the range [2, 2]: [('red', 'apples')]
        :param word_list: List of words which form a sentence when joined in
                          the same order.
        :return: List of contender phrases that are formed after dropping
                 stopwords and punctuations.
        """
        groups = groupby(word_list, lambda x: x not in self.to_ignore)
        phrases = [tuple(group[1]) for group in groups if group[0]]
        return list(
            filter(
                lambda x: self.min_length <= len(x) <= self.max_length, phrases
            )
         )

In [181]:
# rl_list=[]
number_of_asins_on_which_keyword_separators_should_be_applied= 100
counter=0
rl_dict={}
TFIDF_dict={}
vec, map = TF_IDF_all(asin_dict)

for i in asin_dict.keys():
    if counter < number_of_asins_on_which_keyword_separators_should_be_applied:
#         print asin_dict[i]
        c = Rake()
        c.extract_keywords_from_text(asin_dict[i])
        rl_dict[i]= c.rank_list
        counter+=1
        ###########################
        # Run TF-IDF
        ###########################
#         #test_list = [0, 2, 3, 4, 8, 9, 12, 13, 14]
#         test_list = [0]

#         for i in test_list:
#             my_asin = asin_dict.keys()[i]
#             print my_asin
#         print asin_title_map[i]

        TFIDF_dict[i] = TF_IDF_asin(i, asin_dict, vec, map, 10)
#             print keywords
    else:
        break
        #     rl_list.append(c.rank_list)




In [182]:
# d = Rake()
# d.extract_keywords_from_text('I love cats and cats')
# print d.rank_list
# print d.ranked_phrases
print len(TFIDF_dict.keys())


100


In [185]:
#converts reviews to embedding dict where key is the ASIN of a book
count=0
review_dict={}
TFIDF_review_dict={}

lower_cutoff_for_word_to_be_considered=1#model parameter
TFIDF_lower_cutoff_for_word_to_be_considered=0.0#model parameter

number_of_reviews_for_clustering=100#select how many reviews are being used for clustering, small number reduces computation time. 

max_word_per_review= 10#model parameter
TFIDF_max_word_per_review= 10#model parameter


if number_of_reviews_for_clustering > len(rl_dict.keys()):
    print ("number_of_reviews_for_clustering too large!!")
    print("number of reviews= " + str(len(rl_dict.keys()))+ " requested " + str(number_of_reviews_for_clustering))
else:
    for i in rl_dict.keys():
        normalization_factor=0
#         print i
        if count< number_of_reviews_for_clustering :
            print count
            count +=1
    #         print rl_dict[i]
        #     print rl_list[i][0]
            count_of_words_used=0
#             print "entering RAKE"
            for j in rl_dict[i]:
#                 print "entered RAKE"
                if count_of_words_used < max_word_per_review:
                    count_of_words_used+=1
                    if j[0] > lower_cutoff_for_word_to_be_considered:
                        if j[1] not in embeddings_index.keys():
                            for k in j[1].split():
                #                 print ("k=" + k)
                                if k in embeddings_index.keys():
                                    if i not in review_dict.keys():

                                        review_dict[i]= j[0]* embeddings_index[k] 
                                        normalization_factor+=1
                                    else:
                                        review_dict[i]+= j[0]* embeddings_index[k] 
                                        normalization_factor+=1
        #                         else:
        #                             print k
                        else:
                            if j[1] in embeddings_index.keys():
                #                 print ("j=" + j[1])
                    #             review_dict[i]+= rl_list[j][0]* embeddings_index[j][1]
                                if i not in review_dict.keys():
                                    review_dict[i]= j[0]* embeddings_index[j[1]]
                                    normalization_factor+=1
                                else:
                                    review_dict[i]+= j[0]* embeddings_index[j[1]]
                                    normalization_factor+=1
        #                     else:
        #                         print j[1]
#             print ("entering normalization" + str(i))
            if normalization_factor >0:
#                     review_dict[i]= review_dict[i]/ normalization_factor
                [z/normalization_factor for z in review_dict[i]]
#             print ("RAKE done..")

    #TF-IDF converts reviews to embedding dict where key is the ASIN of a book
#             print ("RAKE done..")
            TFIDF_normalization_factor=0
            TFIDF_count_of_words_used=0
#             print TFIDF_dict[i]
#             print "entering TFIDF"
            for j in TFIDF_dict[i]:
                
                if TFIDF_count_of_words_used < max_word_per_review:
                    TFIDF_count_of_words_used+=1
                    if j[0] > TFIDF_lower_cutoff_for_word_to_be_considered:
                        if j[1] not in embeddings_index.keys():
                            for k in j[1].split():
#                                 print ("k=" + k)
                                if k in embeddings_index.keys():
#                                     print ("k=" + k)
                                    if i not in TFIDF_review_dict.keys():
                                        TFIDF_review_dict[i]= j[0]* embeddings_index[k] 
                                        TFIDF_normalization_factor+=1
                                    else:
                                        TFIDF_review_dict[i]+= j[0]* embeddings_index[k] 
                                        TFIDF_normalization_factor+=1
        #                         else:
        #                             print k
                        else:
                            if j[1] in embeddings_index.keys():
#                                 print ("j=" + j[1])
                                if i not in TFIDF_review_dict.keys():
                                    TFIDF_review_dict[i]= j[0]* embeddings_index[j[1]]
                                    TFIDF_normalization_factor+=1
                                else:
                                    TFIDF_review_dict[i]+= j[0]* embeddings_index[j[1]]
                                    TFIDF_normalization_factor+=1
        #                     else:
        #                         print j[1]
#             print ("entering TFIDF Normalization" + str(i))
#             print TFIDF_review_dict
            if TFIDF_normalization_factor >0:
#                     review_dict[i]= review_dict[i]/ normalization_factor
                [z/TFIDF_normalization_factor for z in TFIDF_review_dict[i]]






0


/Users/srishti/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:34: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
/Users/srishti/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:37: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
/Users/srishti/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:76: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


1
2

/Users/srishti/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:90: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
/Users/srishti/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:79: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal



3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [186]:
print len(TFIDF_review_dict.keys())

100


In [187]:
# #TF-IDF converts reviews to embedding dict where key is the ASIN of a book
# count=0
# TFIDF_review_dict={}
# number_of_reviews_for_clustering=5#select how many reviews are being used for clustering, small number reduces computation time. 
# if number_of_reviews_for_clustering > len(TFIDF_dict.keys()):
#     print ("number_of_reviews_for_clustering too large!!")
#     print("number of reviews= " + str(len(rl_dict.keys()))+ " requested " + str(number_of_reviews_for_clustering))
# else:
#     max_word_per_review= 10#model parameter
#     lower_cutoff_for_word_to_be_considered=0.0#model parameter
#     for i in TFIDF_dict.keys():
#         normalization_factor=0
# #         print i
#         if count< number_of_reviews_for_clustering :
#             count +=1
#             count_of_words_used=0
# #             print TFIDF_dict[i]
#             for j in TFIDF_dict[i]:
#                 print j
#                 if count_of_words_used < max_word_per_review:
#                     count_of_words_used+=1
#                     if j[0] > lower_cutoff_for_word_to_be_considered:
#                         if j[1] not in embeddings_index.keys():
#                             for k in j[1].split():
#                                 print ("k=" + k)
#                                 if k in embeddings_index.keys():
#                                     if i not in TFIDF_review_dict.keys():
#                                         TFIDF_review_dict[i]= j[0]* embeddings_index[k] 
#                                         normalization_factor+=1
#                                     else:
#                                         TFIDF_review_dict[i]+= j[0]* embeddings_index[k] 
#                                         normalization_factor+=1
#         #                         else:
#         #                             print k
#                         else:
#                             if j[1] in embeddings_index.keys():
#                                 print ("j=" + j[1])
#                                 if i not in TFIDF_review_dict.keys():
#                                     TFIDF_review_dict[i]= j[0]* embeddings_index[j[1]]
#                                     normalization_factor+=1
#                                 else:
#                                     TFIDF_review_dict[i]+= j[0]* embeddings_index[j[1]]
#                                     normalization_factor+=1
#         #                     else:
#         #                         print j[1]
#                 if normalization_factor >0:
# #                     review_dict[i]= review_dict[i]/ normalization_factor
#                     [z/normalization_factor for z in TFIDF_review_dict[i]]



In [188]:
count =0
for i in review_dict.keys():
    if i not in TFIDF_review_dict.keys():
        print i

In [189]:
review_dict_list=[]
TFIDF_review_dict_list=[]
for i in review_dict.keys():
    review_dict_list.append(review_dict[i])
    TFIDF_review_dict_list.append(TFIDF_review_dict[i])
    
    

In [206]:
#holy K-means
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10, random_state=0).fit_predict(review_dict_list)
TFIDF_kmeans= KMeans(n_clusters=10, random_state=0).fit_predict(TFIDF_review_dict_list)
print kmeans
print TFIDF_kmeans





[0 9 0 0 0 6 2 6 0 0 7 2 0 0 5 9 6 2 6 3 0 0 9 0 6 0 0 6 6 6 6 0 6 6 6 0 0
 6 7 6 0 7 0 0 9 1 9 9 9 8 0 9 9 6 6 6 0 0 0 0 9 7 0 0 6 0 6 9 0 6 6 6 0 6
 0 6 9 0 6 9 7 0 0 0 6 6 0 6 6 0 6 2 6 7 4 0 7 0 1 0]
[4 7 2 4 7 7 6 7 3 0 5 5 7 9 5 5 2 0 2 0 0 0 5 5 0 0 4 0 5 4 0 0 0 4 0 0 7
 0 1 5 5 5 5 7 7 0 0 7 0 5 3 0 5 0 0 7 9 3 3 4 5 4 9 0 2 7 7 5 9 0 0 7 2 5
 3 0 7 0 5 5 5 9 8 7 0 0 3 0 0 3 2 5 4 5 7 0 5 9 5 4]


In [207]:
#matches the embeddings of review to a title
clusteredTitles= collections.defaultdict(list)
for i in range(0, len(kmeans)):
    for asin_value, word2VecList in review_dict.items():
        if (word2VecList == review_dict_list[i]).all():
#             print asin_value
            clusteredTitles[kmeans[i]].append(asin_title_map[asin_value])
            break
print ("RAKE_Clustered Titles")
# print clusteredTitles

TFIDF_clusteredTitles= collections.defaultdict(list)
for i in range(0, len(TFIDF_kmeans)):
    for asin_value, word2VecList in TFIDF_review_dict.items():
        if (word2VecList == TFIDF_review_dict_list[i]).all():
#             print asin_value
            TFIDF_clusteredTitles[TFIDF_kmeans[i]].append(asin_title_map[asin_value])
            break
print ("TFIDF_Clustered Titles")
# print TFIDF_clusteredTitles




RAKE_Clustered Titles
TFIDF_Clustered Titles


In [208]:
for i in clusteredTitles.keys():
    print "RAKE_Cluster Number: " + str(i)+ str(clusteredTitles[i])

for i in TFIDF_clusteredTitles.keys():
    print "TFIDF_Cluster Number: " + str(i)+ str(TFIDF_clusteredTitles[i])


RAKE_Cluster Number: 0['Tweezerman  Nose Hair Clipper', 'NuGo Organic Nutrition Bar, Dark Double Chocolate, 1.76-Ounce Bars (Pack of 12)', 'Firehouse Moustache Wax, Light', 'Now Foods Slippery Elm 400mg, Capsules, 100-Count', 'Trojan Bareskin Lubricated Condoms 10pc (3 Pack)', "Dental Floss Smart Floss Dr. Tung's 30 yd String", 'derma e Psorzema, Natural Relief for Scaling, Flaking, and Itching, 4 Ounce (113 g)', 'Soft Scrub with Bleach Cleanser, 24-Ounce (Pack of 3)', 'Trojan Vibrating Tri-phoria', 'PowerFactor AA Alkaline Battery 24 Pack', 'Diastix Reagent Strips for Urinalysis to test urine Glucose - 50 Strips', 'Dymatize Nutrition BCAA Complex 5050 Powder, 10.7 Ounce', 'Truform 8810,  Compression Stockings, Anti-Embolism, Thigh High, Closed-Toe, 18 mmHg, Beige, X-Large', 'Mighty Bright 87022 Wallet Magnifier w/ LED Light, Silver', 'Aveeno 1% Hydrocortisone Anti-Itch Cream, Maximum Strength, 1-Ounce Tubes (Pack of 4)', 'Terry Naturally Curamin, 60 Caps', 'Source Naturals Essential E

In [198]:
#computation for example in the poster
name='Duracell 80226938 CopperTop Alkaline-Manganese Dioxide Battery with Duralock Power Preserve Technology, AA Size, 1.5V, (Pack of 14)'
for j in asin_title_map.keys():
    if asin_title_map[j] == name:
        print j
        break


B00AZOOW6O


In [202]:
#computation for example in the poster
pos_reviews_B00AZOOW6O = ''
neg_reviews_B00AZOOW6O = ''

for i in data:
    if i['asin'] == 'B00AZOOW6O':
#     asin_dict[i['asin']].append(i['reviewText'])
        if i['overall'] >= 4:
            pos_reviews_B00AZOOW6O+= i['reviewText']

        else:
            neg_reviews_B00AZOOW6O+=i['reviewText']
            

In [203]:
#computation for example in the poster
print "pos_reviews_B00AZOOW6O= " + pos_reviews_B00AZOOW6O 
print "neg_reviews_B00AZOOW6O =" + neg_reviews_B00AZOOW6O
print asin_dict['B00AZOOW6O']

pos_reviews_B00AZOOW6O= I buy Duracell based on previous experience.  The ProCell batteries are what I've used for years and I have had no issues at all.  I bought standard Coppertops because they were on sale and I see no difference thus far.  They seem to last an appropriate amount of time in remotes and kids toys, which is my main usage.  Given that the ProCell batteries are usually cheaper and just as effective, I have to rate the Coppertops 4 stars.  They work great, but not better than a cheaper alternative.Duracell batteries are about as good as any other high quality battery. One aspect I like of these &#34;Duralock&#34; batteries is that they have a shelf life of about 10 years. I can remember years ago a battery would have a shelf life of about 1 year!  Although I have a bunch of alkalines, when they're gone I'm going rechargables! Rechargables such as Eneloop are high quality and VERY cost effective despite their intial $$$.These Alkaline-Manganese Dioxide batteries seem ver

In [205]:
#computation for example in the poster
# rl_list=[]
# number_of_asins_on_which_keyword_separators_should_be_applied= 100
# counter=0
# rl_dict={}
# TFIDF_dict={}
# vec, map = TF_IDF_all(asin_dict)
c = Rake()
c.extract_keywords_from_text(pos_reviews_B00AZOOW6O)
RAKE_pos_reviews_B00AZOOW6O= c.rank_list
print RAKE_pos_reviews_B00AZOOW6O

c = Rake()
c.extract_keywords_from_text(neg_reviews_B00AZOOW6O)
RAKE_neg_reviews_B00AZOOW6O= c.rank_list
print RAKE_neg_reviews_B00AZOOW6O


[(36.0, u'storm season approaching always need x'), (13.5, u'28 pack last year'), (13.029411764705882, u'however getting less batteries'), (12.862745098039216, u'store every time batteries'), (12.862745098039216, u'manganese dioxide batteries seem'), (11.333333333333334, u'seller said &# 34'), (9.0, u'wireless keep board'), (9.0, u'duralock &# 34'), (9.0, u'drain really fast'), (9.0, u'could find locally'), (9.0, u'coppertops 4 stars'), (8.666666666666666, u'battery acid coming'), (8.529411764705882, u'many batteries around'), (8.529411764705882, u'aaa batteries right'), (8.333333333333334, u'difference thus far'), (8.25, u'fine forflash lights'), (8.0, u'usa .&# 34'), (8.0, u'grand kids toys'), (8.0, u'good job duracelli'), (8.0, u'bought standard coppertops'), (7.75, u'buy duracell based'), (7.5, u'remember years ago'), (7.5, u'high quality battery'), (7.0, u'cost effective despite'), (6.779411764705882, u'used duracell batteries'), (6.779411764705882, u'batteries worked fine'), (6.3

In [193]:
###########################
# Helper functions
###########################

# Generate frequency dictionary of all words in all reviews (needed for TF-IDF)
def generateCorpus(X):
    all_words = defaultdict(int)
    for k in range(0,len(X)):
		wordList = re.sub("[^\w]", " ",  X[k]).split() # Clean and split data
		for words in wordList:
			all_words[words.lower()] += 1
    
    return all_words

class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t, 'n') for t in word_tokenize(doc)]

def TF_IDF_all(asin_dict):

    vectorizer = TfidfVectorizer(stop_words="english", analyzer='word', lowercase = True, tokenizer = LemmaTokenizer(), 
				 ngram_range=(1, 2), min_df = 2, max_df = 0.8)

    vec = vectorizer.fit_transform(list(asin_dict.values()))
    word_map=vectorizer.get_feature_names()

    return vec, word_map

def TF_IDF_asin(asin, asin_dict, vectorizer, word_map, n):
    index = asin_dict.keys().index(asin)
    tf = vectorizer[index]
    
    keywords = []
    for col in tf.nonzero()[1]:
        keywords.append((tf[0, col],word_map[col]))

    sorted_keywords = sorted(keywords, key=lambda t: t[1] * -1)

    return sorted_keywords[:n]
